# Exploratory Data Analysis 

In [3]:
# Import Statements
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns

import warnings
warnings.filterwarnings(action = 'ignore')

In [4]:
data = pd.read_csv('salary_data_cleaned.csv')

In [5]:
def title_simplifier(title):
    if 'data scientist' in title.lower():
        return 'data scientist'
    elif 'data engineer' in title.lower():
        return 'data scientist'
    elif 'analyst' in title.lower():
        return 'analyst'
    elif 'machine learning' in title.lower():
        return 'mle'
    elif 'manager' in title.lower():
        return 'manager'
    elif 'director' in title.lower():
        return 'director'
    else:
        return 'na'


def seniority(title):
    if 'sr' in title.lower() or 'senior' in title.lower() or 'lead' in title.lower() or 'principle' in title.lower():
        return 'senior'
    elif 'jr' in title.lower() or 'jr.' in title.lower():
        return 'jr'
    else:
        return 'na'


In [6]:
data['job_simplfier'] = data['Job Title'].apply(title_simplifier)
data['seniority'] = data['Job Title'].apply(seniority)

data.job_simplfier.value_counts(),data.seniority.value_counts()

(data scientist    398
 na                184
 analyst           102
 manager            22
 mle                22
 director           14
 Name: job_simplfier, dtype: int64,
 na        545
 senior    195
 jr          2
 Name: seniority, dtype: int64)

In [7]:
data['job_state'] = data.job_state.apply(lambda x: x.strip() 
                                         if x.strip().lower() != 'los angeles' else 'CA')

data.job_state.value_counts()

CA    152
MA    103
NY     72
VA     41
IL     40
MD     35
PA     33
TX     28
NC     21
WA     21
NJ     17
FL     16
OH     14
TN     13
DC     11
CO     11
WI     10
UT     10
IN     10
AZ      9
MO      9
AL      8
GA      6
DE      6
KY      6
MI      6
CT      5
IA      5
LA      4
OR      4
NE      4
NM      3
KS      3
ID      2
MN      2
RI      1
SC      1
Name: job_state, dtype: int64

In [8]:
# Job description length
data['desc_len'] = data['Job Description'].apply(lambda x: len(x))

In [9]:
# Competitors count
data['num_comp'] = data['Competitors'].apply(lambda x: len(x.split(',')) if x != '-1' else 0)

In [10]:
data.num_comp

0      0
1      0
2      0
3      3
4      3
      ..
737    3
738    3
739    0
740    0
741    0
Name: num_comp, Length: 742, dtype: int64